In [1]:
import numpy as np
np.random.seed(0)
from global_vars import *
from commons import *
                                                                                                                                                                                                                                                                                                                                                            
import pickle as p
import glob 
import json
import subprocess
import getpass
import os
from nibabel.orientations import axcodes2ornt, ornt_transform, inv_ornt_aff, flip_axis
import matplotlib.pyplot as plt

# is_cords_finalized = True

%load_ext autoreload
%autoreload 2

In [ ]:
with open(f'datasets/{dataset}/volumes.txt') as file_handle:
    volumes_to_use = file_handle.read().splitlines()
print(volumes_to_use)

# for idx, i in enumerate(glob.glob(f'{data_dir}/**')):
#     vol = i.split('/')[-1]
#     if vol not in volumes_to_use and idx < 1000:
#         print(vol)
    
#     if idx > 1000:
#         break

In [ ]:
def add_all_labels(paths):
    affine = None
    data = np.zeros((256,256,400))
    for idx, path in enumerate(paths):
        print(path)
        img = nb.load(path)
        if affine is None:
            affine = img.affine
        img_data= img.get_fdata()
        x,y,z = img.shape
        data[:x, :y, :z] += img_data
        
    empty_header = nibabel.Nifti1Header()
    all_label_img = nb.Nifti1Image(data, affine, empty_header)
    return all_label_img

def resize(img, shape=(256, 256, 400), is_label=False):
    if is_label:
        order = 0
    else:
        order = 3
    mode='constant'
    print(img.shape)
    img = resample_from_to(img, [shape, img.affine], order=order, mode=mode, cval=0)
    return img

def crop(paths, shape, img=None):
    s1, e1, s2, e2, s3, e3 = shape
    for path in paths:
        img = nb.load(path) if img is None else img
        img_data= img.get_fdata()
        data = img_data[s1:e1, s2:e2, s3:e3]
        img = nb.Nifti1Image(data, img.affine, img.header)
        save_path = '/'.join(path.split('/')[:-1])
        vol_id = path.split('/')[-1].split('.')[0]
        save_volume(img, f'{save_path}_cropped/{vol_id}')

def remove_black(labelmap):
    clean_labels = []
    start, end = None, None
    start_2, end_2 = None, None
    for i, frame in enumerate(labelmap):
        unique, counts = np.unique(frame, return_counts=True)
        if counts[0] / sum(counts) < .99:
            clean_labels.append(frame)
            if start is None:
                start = i
            elif start_2 is None:
                start_2 = i
            if end is not None:
                end_2 = end
            end = None
        if end is None:
            end = i
    if end is None:
        end = labelmap.shape[0]
            
    return np.array(clean_labels), start, end, start_2, end_2

def remove_black_3channels(data,fat,water,labels):
    clean_data,clean_fat,clean_water, clean_labels = [], [],[],[]
    for i, frame in enumerate(labels):
        unique, counts = np.unique(frame, return_counts=True)
        if counts[0] / sum(counts) < .99:
            clean_labels.append(frame)
            clean_data.append(data[i])
            clean_water.append(water[i])
            clean_fat.append(fat[i])
    return np.array(clean_data), np.array(clean_fat), np.array(clean_water), np.array(clean_labels)

In [ ]:
processed_dir = f'{DEFAULT_OUTPUT_PATH}/KORA/processed'
kora_labels = glob.glob(f'{processed_dir}/label/**')
kora_volumes = glob.glob(f'{processed_dir}/volume/**')
kora_volumes_f = glob.glob(f'{processed_dir}/volume_f/**')
kora_volumes_w = glob.glob(f'{processed_dir}/volume_w/**')
kora_volumes_in = glob.glob(f'{processed_dir}/volume_in/**')

processed_dir = f'{DEFAULT_OUTPUT_PATH}/NAKO/processed'
nako_labels = glob.glob(f'{processed_dir}/label/100112**')
nako_volumes = glob.glob(f'{processed_dir}/volume/100112**')
nako_volumes_f = glob.glob(f'{processed_dir}/volume_f/100112**')
nako_volumes_w = glob.glob(f'{processed_dir}/volume_w/100112**')
nako_volumes_in = glob.glob(f'{processed_dir}/volume_in/100112**')

processed_dir = f'{DEFAULT_OUTPUT_PATH}/UKB/n4_corrected_2/vol/**'
# ukb_labels = glob.glob(f'{processed_dir}/label/**')
ukb_volumes = glob.glob(f'{processed_dir}/OPP_ras_stitched_n4_corrected_sitk.nii.gz')
ukb_volumes_f = glob.glob(f'{processed_dir}/F_ras_stitched_n4_corrected_sitk.nii.gz')
ukb_volumes_w = glob.glob(f'{processed_dir}/W_ras_stitched_n4_corrected_sitk.nii.gz')
ukb_volumes_in = glob.glob(f'{processed_dir}/IN_ras_stitched_n4_corrected_sitk.nii.gz')

ukb1_volumes = glob.glob(f'{processed_dir}/OPP_ras_stitched.nii.gz')
ukb1_volumes_f = glob.glob(f'{processed_dir}/F_ras_stitched.nii.gz')
ukb1_volumes_w = glob.glob(f'{processed_dir}/W_ras_stitched.nii.gz')
ukb1_volumes_in = glob.glob(f'{processed_dir}/IN_ras_stitched.nii.gz')
# processed_dir = f'{DEFAULT_OUTPUT_PATH}/NAKO/processed'
# nako_labels = glob.glob(f'{processed_dir}/label/**')
# nako_volumes = glob.glob(f'{processed_dir}/volume/**')
# processed_dir = f'{DEFAULT_OUTPUT_PATH}/UKB/processed'
# ukb_labels = glob.glob(f'{processed_dir}/label/**')
# ukb_volumes = glob.glob(f'{processed_dir}/volume/**')

# processed_dir = f'{DEFAULT_OUTPUT_PATH}/KORA/n4_corrected_2'
# # kora_labels = glob.glob(f'{processed_dir}/label/**')
# kora_volumes = glob.glob(f'{processed_dir}/vol/**/**_sitk.nii.gz')
# processed_dir = f'{DEFAULT_OUTPUT_PATH}/NAKO/n4_corrected_2'
# # nako_labels = glob.glob(f'{processed_dir}/label/**')
# nako_volumes = glob.glob(f'{processed_dir}/vol/**/**_sitk.nii.gz')
# processed_dir = f'{DEFAULT_OUTPUT_PATH}/UKB/n4_corrected_2'
# # ukb_labels = glob.glob(f'{processed_dir}/label/**')
# ukb_volumes = glob.glob(f'{processed_dir}/vol/**/**_sitk.nii.gz')

# all_labels =  kora_labels+nako_labels+ukb_labels
# all_volumes = kora_volumes + nako_volumes + ukb_volumes

# all_volumes =  kora_volumes + kora_volumes_f + kora_volumes_w +kora_volumes_in + nako_volumes + nako_volumes_f + nako_volumes_w +nako_volumes_in + ukb_volumes + ukb_volumes_f + ukb_volumes_w +ukb_volumes_in
all_ukb = ukb_volumes + ukb_volumes_f + ukb_volumes_w + ukb_volumes_in
all_ukb1 = ukb1_volumes + ukb1_volumes_f + ukb1_volumes_w + ukb1_volumes_in

all_u = all_ukb+all_ukb1
# all_nako = nako_volumes + nako_volumes_f + nako_volumes_w + nako_volumes_in
# print(all_volumes, len(all_volumes))
# print(kora_volumes, kora_labels)
# out of 1534, 1507 volumes got processed, 27 rogue volumes across 3 dataset.
# all_nako

In [ ]:
for vol in all_u:
    print(vol)
    vol_id = vol.split('/')[-1].split('.')[0]
#     vol_id = vol.split('/')[-2]
    save_path = '/'.join(vol.split('/')[:-1])
    print(vol_id, save_path)
    vol_img = nb.load(vol)
    resized_img = resize(vol_img)
    crop([vol], [16, 208, 0, 174, 3, 115], resized_img)
#     save_volume(resized_img, f'{save_path}_resized/{vol_id}')

In [ ]:
for vol in nako_labels:
    vol_id = vol.split('/')[-1].split('.')[0]
    save_path = '/'.join(vol.split('/')[:-1])
    print(vol_id, save_path)
    vol_img = nb.load(vol)
    resized_img = resize(vol_img, is_label=True)
    save_volume(resized_img, f'{save_path}_resized/{vol_id}')

In [ ]:
processed_resized_dir = f'{DEFAULT_OUTPUT_PATH}/KORA/processed'
kora_labels = glob.glob(f'{processed_resized_dir}/label_resized/**')
kora_volumes = glob.glob(f'{processed_resized_dir}/volume_resized/**')
kora_fvolumes = glob.glob(f'{processed_resized_dir}/volume_f_resized/**')
kora_wvolumes = glob.glob(f'{processed_resized_dir}/volume_w_resized/**')
kora_involumes = glob.glob(f'{processed_resized_dir}/volume_in_resized/**')

processed_resized_dir = f'{DEFAULT_OUTPUT_PATH}/NAKO/processed'
nako_labels = glob.glob(f'{processed_resized_dir}/label_resized/100112**')
nako_volumes = glob.glob(f'{processed_resized_dir}/volume_resized/100112**')
nako_fvolumes = glob.glob(f'{processed_resized_dir}/volume_f_resized/100112**')
nako_wvolumes = glob.glob(f'{processed_resized_dir}/volume_w_resized/100112**')
nako_involumes = glob.glob(f'{processed_resized_dir}/volume_in_resized/100112**')

processed_resized_dir = f'{DEFAULT_OUTPUT_PATH}/UKB/processed/unsampled'
ukb_labels = glob.glob(f'{processed_resized_dir}/label_resized/**')
ukb_volumes = glob.glob(f'{processed_resized_dir}/volume_resized/**')
ukb_fvolumes = glob.glob(f'{processed_resized_dir}/volume_f_resized/**')
ukb_wvolumes = glob.glob(f'{processed_resized_dir}/volume_w_resized/**')
ukb_involumes = glob.glob(f'{processed_resized_dir}/volume_in_resized/**')
# processed_resized_dir = f'{DEFAULT_OUTPUT_PATH}/NAKO/processed_resized'
# nako_labels = glob.glob(f'{processed_resized_dir}/label/**')
# nako_volumes = glob.glob(f'{processed_resized_dir}/volume/**')
# processed_resized_dir = f'{DEFAULT_OUTPUT_PATH}/UKB/processed_resized'
# ukb_labels = glob.glob(f'{processed_resized_dir}/label/**')
# ukb_volumes = glob.glob(f'{processed_resized_dir}/volume/**')

# processed_resized_dir = f'{DEFAULT_OUTPUT_PATH}/KORA/n4_corrected_2/vol_resized'
# # kora_labels = glob.glob(f'{processed_resized_dir}/label/**')
# kora_volumes = glob.glob(f'{processed_resized_dir}/volume/**')
# processed_resized_dir = f'{DEFAULT_OUTPUT_PATH}/NAKO/n4_corrected_2/vol_resized'
# # nako_labels = glob.glob(f'{processed_resized_dir}/label/**')
# nako_volumes = glob.glob(f'{processed_resized_dir}/volume/**')
# processed_resized_dir = f'{DEFAULT_OUTPUT_PATH}/UKB/n4_corrected_2/vol_resized'
# # ukb_labels = glob.glob(f'{processed_resized_dir}/label/**')
# ukb_volumes = glob.glob(f'{processed_resized_dir}/volume/**')

In [ ]:
# kora_fvolumes
# nb.load(kora_fvolumes[0]).shape

In [ ]:
# kls = add_all_labels(kora_labels)
# volume_3_view_viewer(get_volume_data(kls))
# save_volume(kls, f'all_labels_merged/all_kora_labels')

# kls = add_all_labels(nako_labels)
# volume_3_view_viewer(get_volume_data(kls))
# save_volume(kls, f'all_labels_merged/all_nako_labels')

# kls = add_all_labels(ukb_labels)
# volume_3_view_viewer(get_volume_data(kls))
# save_volume(kls, f'all_labels_merged/all_ukb_labels')

In [ ]:
# img = nb.load(f'all_labels_merged/all_kora_labels.nii.gz')
# img_data = img.get_fdata()
# img_d1, s1,e1, s12, e12 = remove_black(img_data)
# img_data1 = np.moveaxis(img_data, 1, 0)
# img_d2, s2,e2, s22, e22 = remove_black(img_data1)
# img_data2 = np.moveaxis(img_data, 2, 0)
# img_d3,s3,e3, s32, e32 = remove_black(img_data2)
# # img_data = np.moveaxis(img_data, axis=0)

# print('kora', img_d1.shape, img_d2.shape, img_d3.shape, s1,e1,s2,e2,s3,e3,'---', s12,e12,s22,e22,s32,e32)

# img = nb.load(f'all_labels_merged/all_nako_labels.nii.gz')
# img_data = img.get_fdata()
# img_d1,s1,e1, s12, e12 = remove_black(img_data)
# img_data1 = np.moveaxis(img_data, 1, 0)
# img_d2,s2,e2, s22, e22 = remove_black(img_data1)
# img_data2 = np.moveaxis(img_data, 2, 0)
# img_d3,s3,e3, s32, e32 = remove_black(img_data2)
# # img_data = np.moveaxis(img_data, axis=0)

# print('nako', img_d1.shape, img_d2.shape, img_d3.shape, s1,e1,s2,e2,s3,e3,'---', s12,e12,s22,e22,s32,e32)

# img = nb.load(f'all_labels_merged/all_ukb_labels.nii.gz')
# img_data = img.get_fdata()
# img_d1,s1,e1, s12, e12 = remove_black(img_data)
# img_data1 = np.moveaxis(img_data, 1, 0)
# img_d2,s2,e2, s22, e22 = remove_black(img_data1)
# img_data2 = np.moveaxis(img_data, 2, 0)
# img_d3,s3,e3, s32, e32 = remove_black(img_data2)
# # img_data = np.moveaxis(img_data, axis=0)

# print('ukb', img_d1.shape, img_d2.shape, img_d3.shape, s1,e1, s2,e2, s3,e3,'---', s12,e12,s22,e22,s32,e32)

In [ ]:
# KORA = 158,100,91   ---  49, 206, 23, 122, 84, 176  --> 40, 210, 5, 145, 80, 180 == 170, 140, 100  == 176, 144, 112/96  (Multiples of 16)
#                                                       "40, 215, 5, 175, 70, 180" == "175, 170, 110" ==  '192 176 112'== 32, 224, 2, 178, 69, 181
# NAKO = 158,104,76    --- 44, 201, 40, 143, 169, 244  --> 41, 201, 23, 143, 169, 244 == 160, 120, 75 == 160, 128, 84 =AFTER FAT TALL MATCH= 
#                                                       31, 201, 23, 143, 160, 265 == "170, 120, 105" == '192 176 112' == 20, 212, 0, 176, 157, 269
# UKB= 163, 117, 97    --- 47, 209, 42, 158, 80, 176  --> 35, 215, 25, 170, 215, 315 == 180,145, 100 == 176/192, 144/160, 112 =AFTER FAT TALL[Missing] MATCH= 
#            (Evaluated on 6 merged UKB scans. BH+NOBH) 35, 215, 25, 170, 210, 315 == "180, 145, 105" == '192 176 112' == 29, 221, 10, 186, 72, 184  (for BH Merge only)
 
# KORA sample vol size = 250, 188, 370
# NAKO sample vol size = 226,184,316
# UKB sample vol size = 246, 137, 239
# sample target shape = 256, 256,400


In [ ]:
crop(kora_volumes, [32, 224, 2, 178, 69, 181])
crop(kora_fvolumes, [32, 224, 2, 178, 69, 181])
crop(kora_wvolumes, [32, 224, 2, 178, 69, 181])
crop(kora_involumes, [32, 224, 2, 178, 69, 181])

# crop(kora_volumes, [0, 256, 0, 192, 61, 189])
# crop(kora_fvolumes, [0, 256, 0, 192, 61, 189])
# crop(kora_wvolumes, [0, 256, 0, 192, 61, 189])

In [ ]:
crop(kora_labels, [32, 224, 2, 178, 69, 181])
# crop(kora_labels, [0, 256, 0, 192, 61, 189])

In [ ]:
nako_volumes

In [ ]:
crop(nako_volumes, [20, 212, 0, 176, 157, 269])
crop(nako_fvolumes, [20, 212, 0, 176, 157, 269])
crop(nako_wvolumes, [20, 212, 0, 176, 157, 269])
crop(nako_involumes, [20, 212, 0, 176, 157, 269])

In [ ]:
crop(nako_labels, [20, 212, 0, 176, 157, 269])

In [ ]:
# all_ukb = ukb_volumes + ukb_volumes_f + ukb_volumes_w + ukb_volumes_in
# crop(ukb_volumes, [29, 221, 10, 186, 72, 184])
# crop(ukb_fvolumes, [29, 221, 10, 186, 72, 184])
# crop(ukb_wvolumes, [29, 221, 10, 186, 72, 184])
# crop(ukb_involumes, [29, 221, 10, 186, 72, 184])
# 224, 174, 118    192 176 112
crop(ukb_volumes, [16, 208, 0, 174, 3, 115])
crop(ukb_volumes_f, [16, 208, 0, 174, 3, 115])
crop(ukb_volumes_w, [16, 208, 0, 174, 3, 115])
crop(ukb_volumes_in, [16, 208, 0, 174, 3, 115])

In [ ]:
crop(ukb_labels, [16, 208, 0, 174, 3, 115])

In [6]:
processed_dir = f'{DEFAULT_OUTPUT_PATH}/KORA/processed'
kora_labels_c = sorted(glob.glob(f'{processed_dir}/label_resized_cropped/**'))
kora_volumes_c = sorted(glob.glob(f'{processed_dir}/volume_resized_cropped/**'))
kora_volumes_c_f = sorted(glob.glob(f'{processed_dir}/volume_f_resized_cropped/**'))
kora_volumes_c_w = sorted(glob.glob(f'{processed_dir}/volume_w_resized_cropped/**'))
kora_volumes_c_in = sorted(glob.glob(f'{processed_dir}/volume_in_resized_cropped/**'))

processed_dir = f'{DEFAULT_OUTPUT_PATH}/NAKO/processed'
nako_labels_c = sorted(glob.glob(f'{processed_dir}/label_resized_cropped/100112**'))
nako_volumes_c = sorted(glob.glob(f'{processed_dir}/volume_resized_cropped/100112**'))
nako_volumes_c_f = sorted(glob.glob(f'{processed_dir}/volume_f_resized_cropped/100112**'))
nako_volumes_c_w = sorted(glob.glob(f'{processed_dir}/volume_w_resized_cropped/100112**'))
nako_volumes_c_in = sorted(glob.glob(f'{processed_dir}/volume_in_resized_cropped/100112**'))

processed_dir = f'{DEFAULT_OUTPUT_PATH}/UKB/processed'
ukb_labels_c = sorted(glob.glob(f'{processed_dir}/label_resized_cropped/**'))
ukb_volumes_c = sorted(glob.glob(f'{processed_dir}/volume_resized_cropped/**'))
ukb_volumes_c_f = sorted(glob.glob(f'{processed_dir}/volume_f_resized_cropped/**'))
ukb_volumes_c_w = sorted(glob.glob(f'{processed_dir}/volume_w_resized_cropped/**'))
ukb_volumes_c_in = sorted(glob.glob(f'{processed_dir}/volume_in_resized_cropped/**'))

# processed_dir = f'{DEFAULT_OUTPUT_PATH}/NAKO/processed_resized'
# nako_labels_c = sorted(glob.glob(f'{processed_dir}/label_cropped/**'))
# nako_volumes_c = sorted(glob.glob(f'{processed_dir}/volume_cropped/**'))
# processed_dir = f'{DEFAULT_OUTPUT_PATH}/UKB/processed_resized'
# ukb_labels_c = sorted(glob.glob(f'{processed_dir}/label_cropped/**'))
# ukb_volumes_c = sorted(glob.glob(f'{processed_dir}/volume_cropped/**'))

# processed_dir = f'{DEFAULT_OUTPUT_PATH}/KORA/n4_corrected_2/vol_resized'
# # kora_labels_c = sorted(glob.glob(f'{processed_dir}/label_cropped/**'))
# kora_volumes_c = sorted(glob.glob(f'{processed_dir}/volume_cropped/**'))
# processed_dir = f'{DEFAULT_OUTPUT_PATH}/NAKO/n4_corrected_2/vol_resized'
# # nako_labels_c = sorted(glob.glob(f'{processed_dir}/label_cropped/**'))
# nako_volumes_c = sorted(glob.glob(f'{processed_dir}/volume_cropped/**'))
# processed_dir = f'{DEFAULT_OUTPUT_PATH}/UKB/n4_corrected_2/vol_resized'
# # ukb_labels_c = sorted(glob.glob(f'{processed_dir}/label_cropped/**'))
# ukb_volumes_c = sorted(glob.glob(f'{processed_dir}/volume_cropped/**'))

In [7]:
labels_c = ukb_labels_c+nako_labels_c+kora_labels_c
volumes_c = ukb_volumes_c+ nako_volumes_c + kora_volumes_c
volumes_cf = ukb_volumes_c_f+ nako_volumes_c_f + kora_volumes_c_f
volumes_cw = ukb_volumes_c_w+ nako_volumes_c_w + kora_volumes_c_w
volumes_cin = ukb_volumes_c_in+ nako_volumes_c_in + kora_volumes_c_in



In [ ]:
def visualize_and_save(volid, vol_root=f'{processed_dir}/label_cropped', label_root=f'{processed_dir}/volume_cropped', img_save_path = f'{processed_dir}/merged_imgs'):
    vol = nb.load(f'{vol_root}/{volid}.nii.gz')
    label = nb.load(f'{label_root}/{volid}.nii.gz') if label_root is not None else None

    im = vol.get_fdata()
    x = im.shape[2]//2
    masked = label.get_fdata() if label is not None else None
    plt.figure()
    plt.imshow(np.rot90(im[:,:,x]), 'gray', interpolation='none')
    if masked is not None:
        plt.imshow(np.rot90(masked[:,:,x]), 'jet', interpolation='none', alpha=0.5)
    plt.savefig(f'{img_save_path}/{volid}.png',  dpi=250, quality=95)
    plt.show()

In [ ]:
for p in nako_volumes_c:
    volid = p.split('/')[-1].split('.')[0]
    vol_root= '/'.join(p.split('/')[:-2])+'/volume_resized_cropped'
    label_root= '/'.join(p.split('/')[:-2])+'/label_resized_cropped'
    img_save_path= '/'.join(p.split('/')[:-3])+'/merged_imgs_axial'
    print(volid, vol_root, label_root, img_save_path)
    create_if_not(img_save_path)
    visualize_and_save(volid, vol_root, label_root, img_save_path)

In [ ]:
# NAKO_REFRENCE_IMAGE = '100083 temp/NAKO/processed_resized/volume_cropped temp/NAKO/processed_resized/label_cropped temp/NAKO/merged_imgs'
# KORA_REFERENCE_IMAGE = 'KORA2459745 temp/KORA/processed_resized/volume_cropped temp/KORA/processed_resized/label_cropped temp/KORA/merged_imgs'
# UKB_REFERENCE_IMAGE= '5934622_20201_2_0 temp/UKB/processed_resized/volume_cropped temp/UKB/processed_resized/label_cropped temp/UKB/merged_imgs'
# ALL_REFERENCE_IMAGE='100083 temp/NAKO/processed_resized/volume_cropped temp/NAKO/processed_resized/label_cropped temp/NAKO/merged_imgs'

In [ ]:
# def estimate_weights_mfb(labels, no_of_class=9):
#     class_weights = np.zeros_like(labels)
#     unique, counts = np.unique(labels, return_counts=True)
#     median_freq = np.median(counts)
#     weights = np.zeros(no_of_class)
#     for i, label in enumerate(unique):
#         class_weights += (median_freq // counts[i]) * np.array(lab print('vol:', np.argwhere(np.isnan(img.get_fdata())))
#             hist_matched_img = hist_match(img, reference_path)
#             save_volume(hist_matched_img, f'{new_path}/{phase}/volume_hist{phase_modes[i]}/{vol_id}')
#             hist_matched_img_all = hist_match(img, all_reference_path)
#             save_volume(hist_matched_img_all, f'{all_path}/{phase}/volume_hist{phase_modes[i]}/{vol_id}')
#             print('hist vol:', np.argwhere(np.isnan(hist_matched_img.get_fdata())))
#             print('hist vol_all:', np.argwhere(np.isnan(hist_matched_img_all.get_fdata())))
            
#             intensity_matched_img = intensity_matching(img, reference_path)
#             save_volume(intensity_matched_img, f'{new_path}/{phase}/volume_intensity{phase_modes[i]}/{vol_id}')
#             intensity_matched_img_all = intensity_matching(img, all_reference_path)
#             save_volume(intensity_matched_img_all, f'{all_path}/{phase}/volume_intensity{phase_modes[i]}/{vol_id}') 
#             print('intensity vol:', np.argwhere(np.isnan(intensity_matched_img.get_fdata())))
#             print('intensity vol All:', np.argwhere(np.isnan(intensity_matched_img_all.get_fdata())))els == label)
#         weights[int(label)] = median_freq // counts[i]

#     grads = np.gradient(labels)
#     edge_weights = (grads[0] ** 2 + grads[1] ** 2) > 0
#     class_weights += 2 * edge_weights

#     return class_weights, weights

# def estimate_weights_per_slice(labels, no_of_class=9):
#     weights_per_slice = []
#     for slice_ in labels:
#         unique, counts = np.unique(slice_, return_counts=True)
#         median_freq = np.median(counts)
#         weights = np.zeros(no_of_class)
#         for i, label in enumerate(unique):
#             weights[int(label)] = median_freq // counts[i]
#         weights_per_slice.append(weights)

#     return np.array(weights_per_slice)

# def min_max_norm(img):
# #     return img
#     volume = img.get_fdata()
#     volume = (volume - np.min(volume)) / (np.max(volume) - np.min(volume))
#     return nb.Nifti1Image(volume, img.affine, img.header)
     

In [14]:
# kora_test = ['KORA2456289', 'KORA2458366', 'KORA2459728', 'KORA2459745']
# nako_test = ['100025', '100035', '100085','100083']
# ukb_test = ['1019631_20201_2_0','1013391_20201_2_0','1108214_20201_2_0','1026125_20201_2_0','1036900_20201_2_0']

phase_modes = {
    0: '',
    1: '',
    2: '_f',
    3: '_w',
    4: '_in'
}
references = {
    "KORA": {
        1: f'{DEFAULT_OUTPUT_PATH}/KORA/processed/volume_resized_cropped/KORA2459745.nii.gz',
        2: f'{DEFAULT_OUTPUT_PATH}/KORA/processed/volume_f_resized_cropped/KORA2459745.nii.gz',
        3: f'{DEFAULT_OUTPUT_PATH}/KORA/processed/volume_w_resized_cropped/KORA2459745.nii.gz',
        4: f'{DEFAULT_OUTPUT_PATH}/KORA/processed/volume_in_resized_cropped/KORA2459745.nii.gz',
    },
    "NAKO": {
        1: f'{DEFAULT_OUTPUT_PATH}/NAKO/processed/volume_resized_cropped/100083.nii.gz',
        2: f'{DEFAULT_OUTPUT_PATH}/NAKO/processed/volume_f_resized_cropped/100083.nii.gz',
        3: f'{DEFAULT_OUTPUT_PATH}/NAKO/processed/volume_w_resized_cropped/100083.nii.gz',
        4: f'{DEFAULT_OUTPUT_PATH}/NAKO/processed/volume_in_resized_cropped/100083.nii.gz',
    },
    "UKB": {
        1: f'{DEFAULT_OUTPUT_PATH}/UKB/processed/volume_resized_cropped/1036900_20201_2_0.nii.gz',
        2: f'{DEFAULT_OUTPUT_PATH}/UKB/processed/volume_f_resized_cropped/1036900_20201_2_0.nii.gz',
        3: f'{DEFAULT_OUTPUT_PATH}/UKB/processed/volume_w_resized_cropped/1036900_20201_2_0.nii.gz',
        4: f'{DEFAULT_OUTPUT_PATH}/UKB/processed/volume_in_resized_cropped/1036900_20201_2_0.nii.gz',
    }
}

kora_test_text = '/home/abhijit/Jyotirmay/abdominal_segmentation/quickNAT_pytorch/create_datasets/datasets/kora/test.txt'
nako_test_text = '/home/abhijit/Jyotirmay/abdominal_segmentation/quickNAT_pytorch/create_datasets/datasets/nako/test.txt'
ukb_test_text = '/home/abhijit/Jyotirmay/abdominal_segmentation/quickNAT_pytorch/create_datasets/datasets/ukb/test.txt'

data_path = '/mnt/nas/Abhijit/Jyotirmay/abdominal_segmentation/dataset5'
all_test = []

all_test_paths = [kora_test_text, nako_test_text, ukb_test_text]
for volumes_txt_file in all_test_paths:
    with open(volumes_txt_file) as file_handle:
        volumes_to_use = file_handle.read().splitlines()
        print(volumes_txt_file, volumes_to_use)
        all_test.extend(volumes_to_use)

print(all_test)
exclude_list = ['KORA2460249', '100006','100008', '1004985_20201_2_0','1108214_20201_2_0',
                'KORA2453290','KORA2460734', 'volume']
ngb_list = ['KORA2453290', '1108214_20201_2_0', '1584559_20201_2_0', '1002359_20201_2_0', '6025746_20201_2_0', '4627690_20201_2_0',
            '100183', '100161']
# all_test = kora_test + nako_test + ukb_test
# for ps in zip(labels_c,volumes_c,volumes_cf,volumes_cw,volumes_cin):
# for ps in zip(kora_labels_c, kora_volumes_c, kora_volumes_c_f, kora_volumes_c_w, kora_volumes_c_in):
for ps in zip(nako_labels_c, nako_volumes_c, nako_volumes_c_f, nako_volumes_c_w, nako_volumes_c_in):
# for ps in zip(ukb_labels_c, ukb_volumes_c, ukb_volumes_c_f, ukb_volumes_c_w, ukb_volumes_c_in):
    for i, p in enumerate(ps):
        print(i, p)
#         if i >= 2:
#             continue
        ds = p.split('/')[-4]
        vol_id = p.split('/')[-1].split('.')[0]
        if vol_id not in ngb_list:
            print("continue")
            continue
        print('Processing: ', vol_id, ds)
#         if vol_id in exclude_list:
#             print('excluded:', vol_id)
#             continue
    
        if vol_id == 'volume':
            continue

        if vol_id in all_test:
            phase = 'test'
        else:
            phase = 'train'

        new_path = f'{data_path}/{ds}'
        all_path = f'{data_path}/ALL'
        if i == 0:
            label_img = nb.load(p)
            save_volume(label_img, f'{new_path}/{phase}/label/{vol_id}')
            save_volume(label_img, f'{all_path}/{phase}/label/{vol_id}')
        else:
            reference_path = None
            all_reference_path = references['NAKO'][i]
            reference_path = references[ds][i] if ds in references.keys() else None
            if reference_path == None:
                raise Exception('REFERENCE IS NONE')
            else:
                print(reference_path, all_reference_path)
                
            img = nb.load(p)
            
            save_volume(img, f'{new_path}/{phase}/volume{phase_modes[i]}/{vol_id}')
            save_volume(img, f'{all_path}/{phase}/volume{phase_modes[i]}/{vol_id}')
#             print('vol:', np.argwhere(np.isnan(img.get_fdata())))
            hist_matched_img = hist_match(img, reference_path)
            save_volume(hist_matched_img, f'{new_path}/{phase}/volume_hist{phase_modes[i]}/{vol_id}')
            hist_matched_img_all = hist_match(img, all_reference_path)
            save_volume(hist_matched_img_all, f'{all_path}/{phase}/volume_hist{phase_modes[i]}/{vol_id}')
#             print('hist vol:', np.argwhereUKB(np.isnan(hist_matched_img.get_fdata())))
#             print('hist vol_all:', np.argwhere(np.isnan(hist_matched_img_all.get_fdata())))
            
            intensity_matched_img = intensity_matching(img, reference_path)
            save_volume(intensity_matched_img, f'{new_path}/{phase}/volume_intensity{phase_modes[i]}/{vol_id}')
            intensity_matched_img_all = intensity_matching(img, all_reference_path)
            save_volume(intensity_matched_img_all, f'{all_path}/{phase}/volume_intensity{phase_modes[i]}/{vol_id}') 
#             print('intensity vol:', np.argwhere(np.isnan(intensity_matched_img.get_fdata())))
#             print('intensity vol All:', np.argwhere(np.isnan(intensity_matched_img_all.get_fdata())))

/home/abhijit/Jyotirmay/abdominal_segmentation/quickNAT_pytorch/create_datasets/datasets/kora/test.txt ['KORA2453290', 'KORA2453666', 'KORA2460408']
/home/abhijit/Jyotirmay/abdominal_segmentation/quickNAT_pytorch/create_datasets/datasets/nako/test.txt ['100025', '100035', '100085', '100083']
/home/abhijit/Jyotirmay/abdominal_segmentation/quickNAT_pytorch/create_datasets/datasets/ukb/test.txt ['1019631_20201_2_0', '1013391_20201_2_0', '1108214_20201_2_0', '1026125_20201_2_0', '1036900_20201_2_0']
['KORA2453290', 'KORA2453666', 'KORA2460408', '100025', '100035', '100085', '100083', '1019631_20201_2_0', '1013391_20201_2_0', '1108214_20201_2_0', '1026125_20201_2_0', '1036900_20201_2_0']
0 /mnt/nas/Abhijit/Jyotirmay/abdominal_segmentation/temp2/NAKO/processed/label_resized_cropped/100112.nii.gz
continue
1 /mnt/nas/Abhijit/Jyotirmay/abdominal_segmentation/temp2/NAKO/processed/volume_resized_cropped/100112.nii.gz
continue
2 /mnt/nas/Abhijit/Jyotirmay/abdominal_segmentation/temp2/NAKO/processe

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('/mnt/nas/Users/Sebastian/UKB/ukb_selected_diabetes_data_matched.csv')
df.head()

In [ ]:
print(df['Unnamed: 0'].values.tolist())